In [7]:
import pandas as pd
import selenium
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

# Initialiser le navigateur Web
driver = webdriver.Chrome()

# Accéder au site web d'Auchan
driver.get('https://www.auchan.fr/boutique/promos')  # Remplacez par l'URL exacte du site Auchan

# Attendre que le bouton "Rejeter les cookies" soit visible et cliquer dessus
try:
    reject_cookies_button = WebDriverWait(driver, 40).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Continuer sans accepter')]"))
    )
    reject_cookies_button.click()
    print("Cookies rejetés.")
except Exception as e:
    print("Pas de bouton de rejet des cookies trouvé : ", e)
    

# Wait until the element is clickable
afficher_prix_button = WebDriverWait(driver, 40).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "product-thumbnail__see-prices-button"))
)

driver.execute_script("arguments[0].scrollIntoView(true);", afficher_prix_button)
time.sleep(1)  # Optionally wait a bit after scrolling

driver.execute_script("arguments[0].click();", afficher_prix_button)


# Attendre que la barre de recherche soit visible
search_bar = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "input.shadow--light.journey__search-input.journeySearchInput"))
)

# Taper "67000" dans la barre de recherche
search_bar.send_keys("67000")

# Attendre que les suggestions apparaissent et sélectionner "Strasbourg"
suggestions_list = WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul[role='listbox'] li"))
)

for suggestion in suggestions_list:
    if "Strasbourg" in suggestion.text:
        suggestion.click()
        break

# Pause pour attendre le chargement de la page suivante
time.sleep(5)


# Recherche de l'élément contenant "Auchan Click&collect G.G. Les Halles - Strasbourg"
try:
    store_to_select = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Auchan Click&collect G.G. Les Halles - Strasbourg')]"))
    )
    # Scroller jusqu'à cet élément
    driver.execute_script("arguments[0].scrollIntoView(true);", store_to_select)

    # Chercher directement le bouton "Choisir" associé, en partant de l'élément textuel et descendant dans la hiérarchie
    choose_button = store_to_select.find_element(By.XPATH, ".//following::button[contains(@class, 'btnJourneySubmit')]")
    
    # Cliquer sur le bouton "Choisir"
    driver.execute_script("arguments[0].click();", choose_button)
    
    print("Bouton 'Choisir' cliqué.")
except Exception as e:
    print(f"Erreur lors de la sélection du magasin : {e}")


Cookies rejetés.
Bouton 'Choisir' cliqué.


In [ ]:
# Listes pour stocker les informations
product_names = []
product_prices = []
product_units = []
discount_percentages = []

# URL de base pour la boutique Auchan avec les promotions
base_url = "https://www.auchan.fr/boutique/promos?page="
page_num = 1  # Commencer à la première page

# Boucle pour scraper toutes les pages
while True:
    print(f"Scraping page {page_num}")
    driver.get(base_url + str(page_num))  # Charger la page avec le numéro actuel

    # Attendre que la page se charge complètement
    time.sleep(3)
    
    # Récupérer les produits sur la page actuelle
    products = driver.find_elements(By.XPATH, "//article[contains(@class, 'product-thumbnail')]")
    
    if not products:
        print(f"Aucun produit trouvé sur la page {page_num}. Fin de la pagination.")
        break
    
    print(f"Nombre de produits trouvés sur la page {page_num}: {len(products)}")
    
    # Récupérer les informations des produits sur la page actuelle
    for p in products:  
        # Récupérer le nom du produit
        try:
            name_element = p.find_element(By.CLASS_NAME, "product-thumbnail__description")
            product_names.append(name_element.text.strip())
        except Exception as e:
            print(f"Erreur lors de la récupération du nom du produit : {e}")
            product_names.append(None)

        # Récupérer le prix du produit
        try:
            price_element = p.find_element(By.CLASS_NAME, "product-thumbnail__price")
            product_prices.append(price_element.text.strip())
        except Exception as e:
            print(f"Erreur lors de la récupération du prix : {e}")
            product_prices.append(None)

        # Récupérer l'unité du produit (g, kg, L, etc.)
        try:
            unit_element = p.find_element(By.CLASS_NAME, "product-attribute")
            product_units.append(unit_element.text.strip())
        except Exception as e:
            print(f"Erreur lors de la récupération de l'unité : {e}")
            product_units.append(None)

        # Récupérer le pourcentage de réduction
        try:
            discount_element = p.find_element(By.CLASS_NAME, "product-discount-label")
            discount_percentages.append(discount_element.text.strip())
        except Exception as e:
            print(f"Erreur lors de la récupération du pourcentage de réduction : {e}")
            discount_percentages.append(None)

    # Passer à la page suivante
    page_num += 1

    # Optionnel : Limiter à un certain nombre de pages (ici, par exemple, 10 pages)
    # if page_num > 10:
    #     break

# Fermer le driver après le scraping
#driver.quit()

# Sauvegarder les informations extraites dans un fichier CSV
with open('Auchan_products.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Nom du Produit', 'Prix', 'Unité', 'Pourcentage de Réduction'])
    for i in range(len(product_names)):
        writer.writerow([product_names[i], product_prices[i], product_units[i], discount_percentages[i]])

print("Scraping terminé et fichier CSV créé avec succès.")


Scraping page 1
Nombre de produits trouvés sur la page 1: 14
Erreur lors de la récupération du pourcentage de réduction : Message: no such element: Unable to locate element: {"method":"css selector","selector":".product-discount-label"}
  (Session info: chrome=130.0.6723.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF77E413AB5+28005]
	(No symbol) [0x00007FF77E3783B0]
	(No symbol) [0x00007FF77E21580A]
	(No symbol) [0x00007FF77E265A3E]
	(No symbol) [0x00007FF77E265D2C]
	(No symbol) [0x00007FF77E25937C]
	(No symbol) [0x00007FF77E28BA7F]
	(No symbol) [0x00007FF77E259246]
	(No symbol) [0x00007FF77E28BC50]
	(No symbol) [0x00007FF77E2AB8B3]
	(No symbol) [0x00007FF77E28B7E3]
	(No symbol) [0x00007FF77E2575C8]
	(No symbol) [0x00007FF77E258731]
	GetHandleVerifier [0x00007FF77E70643D+3118829]
	GetHandleVerifier [0x00007FF77E756C90+3448640]
	GetHandleVerifi

In [9]:
df = pd.DataFrame({
    'Nom': product_names,
    'Prix': product_prices,
    'Unité': product_units,
    'Réduction': discount_percentages,
    
})

df

,Nom,Prix,Unité,Réduction
0,KREMA Red Dingue Bonbons aux fruits rouges,"5,32€\n3,51€",580g,None
1,HARIBO Dragibus aux couleurs d'Halloween,"2,31€",250g,3 achetés = 2 payés
2,AUCHAN LE CHARCUTIER Chiffonnade de Coppa,"3,69€",80g,-50% sur le 2ème
3,AUCHAN LE CHARCUTIER Chiffonnade de jambon ser...,"3,59€",80g,-50% sur le 2ème
4,AUCHAN LE CHARCUTIER Chiffonnade de rosette,"3,59€",80g,-50% sur le 2ème
...,...,...,...,...
8973,Bananes Prix Bas,"0,99€",5 pièces,None
8974,AUCHAN CULTIVONS LE BON thon listao entier au ...,"2,23€",140g,None
8975,POUCE Œufs de poules,"1,03€",6 œufs,None
8976,POKEMON Mug Pikachu 320ml - Noir,"14,00€",None,None


In [11]:
# Sauvegarder les données dans un fichier Excel
df.to_csv("auchan_products.csv", index=False)

print("Données enregistrées dans auchan_products.csv")

Données enregistrées dans auchan_products.csv
